In [8]:
# Imports
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
import datetime
import time
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras import regularizers

np.random.seed(7)

# Constants
TRAIN_AND_VALIDATION_TEST_SPLIT = 0.3
VALID_AND_TEST_SPLIT            = 0.5
BATCH_SIZE                      = 10

relevent_attributes=["Priority","RaisedByID","AssignedToID","StatusCode",\
                     "ProjectCode","Category","SubCategory","HoursEstimate","HoursActual"]
X_attributes=["Priority","RaisedByID","AssignedToID","StatusCode",\
                     "ProjectCode","Category","SubCategory","HoursEstimate"]
Y_attributes=["HoursActual"]

%matplotlib inline  

In [9]:
def set_curr_time():
    dt = datetime.datetime.now()
    curr_dt = '{0}{1}{2}_{3}_{4}'.format(datetime.datetime.now().year, datetime.datetime.now().month,
                                       datetime.datetime.now().day, datetime.datetime.now().hour,
                                      datetime.datetime.now().minute)
    return curr_dt

## Machine Learning

In [10]:
data = pd.read_csv('../SiP_dataset-master/SIP_CAT.csv', encoding='cp1252') 
data.head()

,Unnamed: 0,Priority,RaisedByID,AssignedToID,AuthorisedByID,StatusCode,ProjectCode,Category,SubCategory,HoursEstimate,HoursActual
0,0,-0.683936,1.052282,1.099895,-2.113198,1.040642,-0.142837,-0.651337,-0.315307,0.133422,-0.166257
1,1,-0.683936,1.052282,0.249867,-2.113198,1.040642,-0.142837,-0.651337,-0.315307,-0.109292,-0.089862
2,2,-0.102534,-1.515859,1.099895,-2.113198,1.040642,-0.142837,1.995546,-0.001708,-0.327734,-0.181536
3,3,1.641673,0.649437,0.249867,-2.113198,1.040642,-0.142837,-0.651337,-1.256102,-0.327734,-0.181536
4,4,4.548685,0.448014,-1.290809,-2.113198,1.040642,-0.142837,-0.651337,-1.256102,-0.230648,-0.140792


In [11]:
data = data[relevent_attributes]
ROW_COUNT = max(data.count())

In [12]:
# HoursEstimate_filter=HoursEstimate[HoursEstimate["HoursEstimate"]<200]
# HoursEstimate_filter=HoursEstimate_filter[HoursEstimate_filter["HoursActual"]<800]

x_train_label, xx_test_label, y_train_label, yy_test_label \
              = train_test_split(data[X_attributes].values[:], data[Y_attributes].values[:], test_size=TRAIN_AND_VALIDATION_TEST_SPLIT)
x_valid_label, x_test_label, y_valid_label, y_test_label \
              = train_test_split(xx_test_label, yy_test_label, test_size=VALID_AND_TEST_SPLIT)  

print("Train examples: {}".format(x_train_label.shape))
print("Valid examples: {}".format(x_valid_label.shape))
print("Test examples: {}".format(x_test_label.shape))

Train examples: (8609, 8)
Valid examples: (1845, 8)
Test examples: (1845, 8)


In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(2048, kernel_initializer='normal'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(1024,  kernel_initializer='normal'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(512, kernel_initializer='normal'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(256, kernel_initializer='normal'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(128, kernel_initializer='normal'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(64, kernel_initializer='normal'), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.488),
    tf.keras.layers.Dense(1)
    
    
#     tf.keras.layers.Dense(1024, activation=tf.nn.relu),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(512, activation=tf.nn.relu),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(256, activation=tf.nn.relu),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(128, activation=tf.nn.relu),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(64, activation=tf.nn.relu),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(1)
])

In [20]:

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

curr_dt = set_curr_time()
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='../logs/{0}'.format(curr_dt), histogram_freq=0, write_graph=True)

tic = time.time()
model.fit(
    x_train_label,
    y_train_label,
    epochs = 300,
    batch_size = BATCH_SIZE,
    validation_data = (x_valid_label, y_valid_label),
    shuffle = True,
#     steps_per_epoch = round(ROW_COUNT/BATCH_SIZE),
    callbacks=[tb_callback])
toc = time.time()

print('Training time:{}'.format(toc-tic))

# model.save('../models/{0}.h5'.format(curr_dt))


Train on 8609 samples, validate on 1845 samples
Epoch 1/300
8609/8609 [==============================] - 8s 909us/step - loss: 1.0950 - mean_absolute_error: 0.1959 - val_loss: 0.9758 - val_mean_absolute_error: 0.2062
Epoch 2/300
8609/8609 [==============================] - 7s 765us/step - loss: 1.0752 - mean_absolute_error: 0.1958 - val_loss: 0.9416 - val_mean_absolute_error: 0.1921
Epoch 3/300
8609/8609 [==============================] - 6s 754us/step - loss: 1.0482 - mean_absolute_error: 0.1923 - val_loss: 0.9384 - val_mean_absolute_error: 0.1773
Epoch 4/300
8609/8609 [==============================] - 7s 760us/step - loss: 1.0402 - mean_absolute_error: 0.1869 - val_loss: 1.0495 - val_mean_absolute_error: 0.4150
Epoch 5/300
8609/8609 [==============================] - 7s 762us/step - loss: 1.0407 - mean_absolute_error: 0.1934 - val_loss: 0.9254 - val_mean_absolute_error: 0.2078
Epoch 6/300
8609/8609 [==============================] - 7s 759us/step - loss: 1.0389 - mean_absolute_error

Epoch 49/300
8609/8609 [==============================] - 7s 777us/step - loss: 0.4199 - mean_absolute_error: 0.1586 - val_loss: 0.7926 - val_mean_absolute_error: 0.1778
Epoch 50/300
8609/8609 [==============================] - 7s 774us/step - loss: 0.5233 - mean_absolute_error: 0.1622 - val_loss: 0.5827 - val_mean_absolute_error: 0.3922
Epoch 51/300
8609/8609 [==============================] - 7s 771us/step - loss: 0.3709 - mean_absolute_error: 0.1598 - val_loss: 0.9070 - val_mean_absolute_error: 0.1710
Epoch 52/300
8609/8609 [==============================] - 7s 776us/step - loss: 0.7083 - mean_absolute_error: 0.1645 - val_loss: 0.8298 - val_mean_absolute_error: 0.1913
Epoch 53/300
8609/8609 [==============================] - 7s 778us/step - loss: 0.7357 - mean_absolute_error: 0.1861 - val_loss: 0.4411 - val_mean_absolute_error: 0.1596
Epoch 54/300
8609/8609 [==============================] - 7s 766us/step - loss: 0.3863 - mean_absolute_error: 0.1565 - val_loss: 0.4029 - val_mean_abs

8609/8609 [==============================] - 7s 767us/step - loss: 0.3723 - mean_absolute_error: 0.1356 - val_loss: 0.2363 - val_mean_absolute_error: 0.1424
Epoch 98/300
8609/8609 [==============================] - 7s 769us/step - loss: 0.2640 - mean_absolute_error: 0.1325 - val_loss: 0.2715 - val_mean_absolute_error: 0.2055
Epoch 99/300
8609/8609 [==============================] - 7s 772us/step - loss: 0.3115 - mean_absolute_error: 0.1372 - val_loss: 0.2182 - val_mean_absolute_error: 0.1383
Epoch 100/300
8609/8609 [==============================] - 7s 758us/step - loss: 0.3284 - mean_absolute_error: 0.1360 - val_loss: 0.2317 - val_mean_absolute_error: 0.1349
Epoch 101/300
8609/8609 [==============================] - 7s 774us/step - loss: 0.2428 - mean_absolute_error: 0.1308 - val_loss: 0.2351 - val_mean_absolute_error: 0.1317
Epoch 102/300
8609/8609 [==============================] - 7s 774us/step - loss: 0.2235 - mean_absolute_error: 0.1232 - val_loss: 0.2391 - val_mean_absolute_erro

8609/8609 [==============================] - 7s 774us/step - loss: 0.2520 - mean_absolute_error: 0.1247 - val_loss: 0.2480 - val_mean_absolute_error: 0.2050
Epoch 146/300
8609/8609 [==============================] - 7s 772us/step - loss: 0.2430 - mean_absolute_error: 0.1222 - val_loss: 0.2344 - val_mean_absolute_error: 0.1383
Epoch 147/300
8609/8609 [==============================] - 7s 769us/step - loss: 0.2440 - mean_absolute_error: 0.1269 - val_loss: 0.2357 - val_mean_absolute_error: 0.1283
Epoch 148/300
8609/8609 [==============================] - 7s 768us/step - loss: 0.3263 - mean_absolute_error: 0.1258 - val_loss: 0.2714 - val_mean_absolute_error: 0.1303
Epoch 149/300
8609/8609 [==============================] - 7s 765us/step - loss: 0.2251 - mean_absolute_error: 0.1267 - val_loss: 0.2497 - val_mean_absolute_error: 0.1320
Epoch 150/300
8609/8609 [==============================] - 7s 760us/step - loss: 0.3378 - mean_absolute_error: 0.1255 - val_loss: 0.2763 - val_mean_absolute_er

Epoch 193/300
8609/8609 [==============================] - 6s 750us/step - loss: 0.2003 - mean_absolute_error: 0.1113 - val_loss: 0.2500 - val_mean_absolute_error: 0.1378
Epoch 194/300
8609/8609 [==============================] - 7s 767us/step - loss: 0.2004 - mean_absolute_error: 0.1148 - val_loss: 0.2186 - val_mean_absolute_error: 0.1220
Epoch 195/300
8609/8609 [==============================] - 7s 764us/step - loss: 0.2815 - mean_absolute_error: 0.1243 - val_loss: 0.4166 - val_mean_absolute_error: 0.1341
Epoch 196/300
8609/8609 [==============================] - 7s 756us/step - loss: 0.2440 - mean_absolute_error: 0.1244 - val_loss: 0.2270 - val_mean_absolute_error: 0.1258
Epoch 197/300
8609/8609 [==============================] - 6s 754us/step - loss: 0.2102 - mean_absolute_error: 0.1151 - val_loss: 0.2150 - val_mean_absolute_error: 0.1270
Epoch 198/300
8609/8609 [==============================] - 6s 752us/step - loss: 0.2021 - mean_absolute_error: 0.1114 - val_loss: 0.2144 - val_me

Epoch 241/300
8609/8609 [==============================] - 7s 766us/step - loss: 0.2348 - mean_absolute_error: 0.1115 - val_loss: 0.2260 - val_mean_absolute_error: 0.1240
Epoch 242/300
8609/8609 [==============================] - 7s 760us/step - loss: 0.2875 - mean_absolute_error: 0.1175 - val_loss: 0.2258 - val_mean_absolute_error: 0.1315
Epoch 243/300
8609/8609 [==============================] - 7s 764us/step - loss: 0.1771 - mean_absolute_error: 0.1070 - val_loss: 0.2637 - val_mean_absolute_error: 0.1329
Epoch 244/300
8609/8609 [==============================] - 7s 769us/step - loss: 0.2006 - mean_absolute_error: 0.1124 - val_loss: 0.2503 - val_mean_absolute_error: 0.1270
Epoch 245/300
8609/8609 [==============================] - 7s 771us/step - loss: 0.2005 - mean_absolute_error: 0.1143 - val_loss: 0.2256 - val_mean_absolute_error: 0.1685
Epoch 246/300
8609/8609 [==============================] - 7s 770us/step - loss: 0.2047 - mean_absolute_error: 0.1124 - val_loss: 0.2305 - val_me

Epoch 289/300
8609/8609 [==============================] - 7s 772us/step - loss: 0.1895 - mean_absolute_error: 0.1077 - val_loss: 0.2036 - val_mean_absolute_error: 0.1360
Epoch 290/300
8609/8609 [==============================] - 7s 771us/step - loss: 0.2029 - mean_absolute_error: 0.1092 - val_loss: 0.2119 - val_mean_absolute_error: 0.1278
Epoch 291/300
8609/8609 [==============================] - 7s 770us/step - loss: 0.1912 - mean_absolute_error: 0.1022 - val_loss: 0.2264 - val_mean_absolute_error: 0.1196
Epoch 292/300
8609/8609 [==============================] - 7s 770us/step - loss: 0.1870 - mean_absolute_error: 0.1011 - val_loss: 0.2134 - val_mean_absolute_error: 0.1304
Epoch 293/300
8609/8609 [==============================] - 7s 771us/step - loss: 0.1881 - mean_absolute_error: 0.1031 - val_loss: 0.2063 - val_mean_absolute_error: 0.1589
Epoch 294/300
8609/8609 [==============================] - 7s 771us/step - loss: 0.1916 - mean_absolute_error: 0.1075 - val_loss: 0.2069 - val_me

In [22]:
pred_y=model.predict(x_test_label)

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             multiple                  18432     
_________________________________________________________________
batch_normalization_12 (Batc multiple                  8192      
_________________________________________________________________
activation_12 (Activation)   multiple                  0         
_________________________________________________________________
dropout_12 (Dropout)         multiple                  0         
_________________________________________________________________
dense_15 (Dense)             multiple                  2098176   
_________________________________________________________________
batch_normalization_13 (Batc multiple                  4096      
_________________________________________________________________
activation_13 (Activation)   multiple                  0         
__________

In [26]:
# #accuracy max-min
# ERROE_MERGE = 0.5 # in hours
# SCALE = (ERROE_MERGE-0.01)/(910 - 0.01)
# correct = 0
# for i in range(0,len(y_test_label)):    
#     if (((pred_y[i] - SCALE) < y_test_label[i][0]) and ((pred_y[i] + SCALE) > y_test_label[i][0])):
#         correct += 1
# print(correct/len(y_test_label))


#accuracy Gussian(mean,std)
ERROE_MERGE = 0.5 # in hours
correct = 0
for i in range(0,len(y_test_label)):
    scale_pred = (pred_y[i]*68.72187295)+(+13.17546792)
    scale_test = (y_test_label[i][0]*68.72187295)+(+13.17546792)

    if (((scale_pred - ERROE_MERGE) < scale_test) and ((scale_pred + ERROE_MERGE) > scale_test)):
        correct += 1
print(correct/len(y_test_label))

0.09051490514905149


# ML - OneHot data

In [4]:
data = pd.read_csv('../SiP_dataset-master/SIP_OneHot.csv', encoding='cp1252') 
data.head()

,Unnamed: 0,Priority,RaisedByID,AssignedToID,AuthorisedByID,HoursEstimate,HoursActual,StatusCode_AUTHORISE,StatusCode_CANCELLED,StatusCode_CHRONICLE,...,SubCategory_Project Management,SubCategory_Release,SubCategory_Research,SubCategory_Staff Management,SubCategory_Staff Recruitment,SubCategory_Support,SubCategory_Technical Specification,SubCategory_Testing,SubCategory_Third Party,SubCategory_Training
0,0,1,58,58,6.0,14.0,1.75,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,58,42,6.0,7.0,7.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,7,58,6.0,0.7,0.70,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,5,50,42,6.0,0.7,0.70,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,10,46,13,6.0,3.5,3.50,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# HoursEstimate_filter=HoursEstimate[HoursEstimate["HoursEstimate"]<200]
# HoursEstimate_filter=HoursEstimate_filter[HoursEstimate_filter["HoursActual"]<800]

x_train_label, xx_test_label, y_train_label, yy_test_label \
              = train_test_split(data.loc[:, data.columns != 'HoursActual'].values[:], data[Y_attributes].values[:], test_size=TRAIN_AND_VALIDATION_TEST_SPLIT)
x_valid_label, x_test_label, y_valid_label, y_test_label \
              = train_test_split(xx_test_label, yy_test_label, test_size=VALID_AND_TEST_SPLIT)  

print("Train examples: {}".format(x_train_label.shape))
print("Valid examples: {}".format(x_valid_label.shape))
print("Test examples: {}".format(x_test_label.shape))

Train examples: (8609, 61)
Valid examples: (1845, 61)
Test examples: (1845, 61)


In [5]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x_train_label.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               7936      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 172,801
Trainable params: 172,801
Non-trainable params: 0
_________________________________________________________________


In [7]:
curr_dt = set_curr_time()
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='../logs/{0}'.format(curr_dt), histogram_freq=0, write_graph=True)

tic = time.time()
NN_model.fit(
    x_train_label,
    y_train_label,
    epochs = 300,
    batch_size = BATCH_SIZE,
    validation_data = (x_valid_label, y_valid_label),
    shuffle = True,
    callbacks=[tb_callback])
toc = time.time()

print('Training time:{}'.format(toc-tic))
pred_y=model.predict(x_test_label)


Train on 8609 samples, validate on 1845 samples
Epoch 1/300
8609/8609 [==============================] - 3s 336us/step - loss: 12.4008 - mean_absolute_error: 12.4008 - val_loss: 11.6302 - val_mean_absolute_error: 11.6302
Epoch 2/300
8609/8609 [==============================] - 2s 261us/step - loss: 12.3496 - mean_absolute_error: 12.3496 - val_loss: 11.6875 - val_mean_absolute_error: 11.6875
Epoch 3/300
8609/8609 [==============================] - 2s 261us/step - loss: 12.3439 - mean_absolute_error: 12.3439 - val_loss: 11.6336 - val_mean_absolute_error: 11.6336
Epoch 4/300
8609/8609 [==============================] - 2s 263us/step - loss: 12.3498 - mean_absolute_error: 12.3498 - val_loss: 11.6245 - val_mean_absolute_error: 11.6245
Epoch 5/300
8609/8609 [==============================] - 2s 261us/step - loss: 12.3487 - mean_absolute_error: 12.3487 - val_loss: 11.6286 - val_mean_absolute_error: 11.6286
Epoch 6/300
8609/8609 [==============================] - 2s 254us/step - loss: 12.3479 

Epoch 48/300
8609/8609 [==============================] - 2s 241us/step - loss: 12.3361 - mean_absolute_error: 12.3361 - val_loss: 11.6437 - val_mean_absolute_error: 11.6437
Epoch 49/300
8609/8609 [==============================] - 2s 241us/step - loss: 12.3384 - mean_absolute_error: 12.3384 - val_loss: 11.6323 - val_mean_absolute_error: 11.6323
Epoch 50/300
8609/8609 [==============================] - 2s 241us/step - loss: 12.3380 - mean_absolute_error: 12.3380 - val_loss: 11.6354 - val_mean_absolute_error: 11.6354
Epoch 51/300
8609/8609 [==============================] - 2s 243us/step - loss: 12.3388 - mean_absolute_error: 12.3388 - val_loss: 11.6216 - val_mean_absolute_error: 11.6216
Epoch 52/300
8609/8609 [==============================] - 2s 242us/step - loss: 12.3375 - mean_absolute_error: 12.3375 - val_loss: 11.6227 - val_mean_absolute_error: 11.6227
Epoch 53/300
8609/8609 [==============================] - 2s 242us/step - loss: 12.3392 - mean_absolute_error: 12.3392 - val_loss:

KeyboardInterrupt: 

# Different models

In [ ]:
model = RandomForestRegressor()
model.fit(train_X,train_y)

# Get the mean absolute error on the validation data
predicted_prices = model.predict(val_X)
MAE = mean_absolute_error(val_y , predicted_prices)
print('Random forest validation MAE = ', MAE)